In [3]:
import pandas as pd
import numpy as np

In [4]:
employee = pd.read_csv("../data/employee_cleaned.csv")

In [6]:
employee.isna().sum()

Unnamed: 0                   0
Education                    0
JoiningYear                  0
City                         0
PaymentTier                  0
Age                          0
Gender                       0
EverBenched                  0
ExperienceInCurrentDomain    0
LeaveOrNot                   0
dtype: int64

In [7]:
employee.shape

(2764, 10)